In [1]:
import torch

<h1>AutoGrad</h1>

In [2]:
x = torch.randn(3,requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([-1405.4962,   192.4249,  -197.8437], grad_fn=<MulBackward0>)


In [3]:
v = torch.tensor([0.1,1.0,0.0001],dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


对y调用backward，实际上对于之前每一步计算都记录了一个雅可比矩阵

In [17]:
x = torch.Tensor([1,2,3])
x.requires_grad_(True)
print(x)

tensor([1., 2., 3.], requires_grad=True)


In [18]:
y = 2 * x
z = 3 * y

In [19]:
v = torch.tensor([1,1,1],dtype=torch.float)
z.backward(v)
#print(y.grad)
print(x.grad)

tensor([6., 6., 6.])


<h1>神经网络</h1>

In [71]:
import torch.nn as nn
import torch.nn.functional as F

In [82]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        #Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [83]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [84]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [85]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0935,  0.0639, -0.0076, -0.0184,  0.0623, -0.1507, -0.0867,  0.0754,
          0.0080,  0.1036]], grad_fn=<AddmmBackward>)


In [86]:
net.zero_grad()
out.backward(torch.randn(1,10),retain_graph=True)

In [87]:
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(out,target)
print(loss)

tensor(1.0812, grad_fn=<MseLossBackward>)


In [88]:
print(loss.grad_fn)

In [89]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0112, -0.0013,  0.0211, -0.0022,  0.0069,  0.0038])


In [92]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [93]:
print(net.conv1.bias.grad)

tensor([-0.0112, -0.0013,  0.0211, -0.0022,  0.0069,  0.0038])
